In [ ]:
import pandas as pd
import numpy as np

In [ ]:
matches = pd.read_csv("results.csv")

In [ ]:
matches.head()

In [ ]:
matches.shape

In [ ]:
winner = []
for i in range (len(matches['home_team'])):
    if matches ['home_score'][i] > matches ['away_score'][i]:
        winner.append(matches['home_team'][i])
    elif matches['home_score'][i] < matches['away_score'][i]:
        winner.append(matches['away_team'][i])
    else:
        winner.append('Draw')
matches['winning_team'] = winner


matches['goal_difference'] = np.absolute(matches['home_score']-matches['away_score'])

matches.head()

In [ ]:
#aataframe francia
df = matches[(matches['home_team'] == 'France') | (matches['away_team'] == 'France')]
france = df.iloc[:]
france.head()

In [ ]:
#creamos una columna para el año
year = []
for row in france['date']:
    year.append(int(row[:4]))
france['match_year'] = year
france_1930 = france[france.match_year >= 1930]
france_1930.count()

In [ ]:
#cual es el resultado usual para Francia?
wins = []
for row in france_1930['winning_team']:
    if row != 'France' and row != 'Draw':
        wins.append('Loss')
    else:
        wins.append(row)

winsdf = pd.DataFrame(wins, columns=['France_Results'])



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
fig, ax = plt.subplots(1)
fig.set_size_inches(10.7, 6.27)
sns.set(style='darkgrid')
sns.countplot(x='France_Results', data=winsdf)

In [ ]:
#agregamos los países que participan en la copa del mundo 2022 qatar
worldcup_teams = ['Cameroon', 'Ghana', 'Morocco', 'Senegal', 'Tunisia', 'Iran', 'Japan', 'South Korea', 'Qatar', 'Saudi Arabia', 'Australia', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Netherlands', 'Poland', 'Portugal', 'Serbia', 'Spain', 'Switzerland', 'Wales', 'Canada', 'Mexico', 'United States', 'Costa Rica', 'Argentina', 'Brazil', 'Ecuador', 'Uruguay']

In [ ]:
df_teams_home = matches[matches['home_team'].isin(worldcup_teams)]
df_teams_away = matches[matches['away_team'].isin(worldcup_teams)]

In [ ]:
df_teams = pd.concat((df_teams_home, df_teams_away))
df_teams.drop_duplicates()
df_teams.count()

In [ ]:
#filtramos partidos desde 1930
year = []
for row in df_teams['date']:
    year.append(int(row[:4]))
df_teams['match_year'] = year
df_teams_1930 = df_teams[df_teams.match_year >= 1930]
df_teams_1930.head()

In [ ]:
#eliminamos columnas que no nos sirven
df_teams_1930 = df_teams_1930.drop(['date', 'home_score', 'away_score', 'tournament', 'city', 'country', 'goal_difference', 'match_year'], axis=1)
df_teams_1930.head()

In [ ]:
#construcción del modelo
#la columna de predicción mostrará 2 si el equipo de casa ganó, 1 si fue empate y 0 si ganó el visitante

df_teams_1930 = df_teams_1930.reset_index(drop=True)
df_teams_1930.loc[df_teams_1930.winning_team == df_teams_1930.home_team, 'winning_team'] = 2
df_teams_1930.loc[df_teams_1930.winning_team == 'Draw', 'winning_team'] = 1
df_teams_1930.loc[df_teams_1930.winning_team == df_teams_1930.away_team, 'winning_team'] = 0

df_teams_1930.head()


In [ ]:
#convertimos los equipos de casa y visitantes de variables categoricas a inputs continuos
#obtenemos variables dummy
final = pd.get_dummies(df_teams_1930, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

In [ ]:
#seáramos x de ys
X = final.drop(['winning_team'], axis=1)
y = final["winning_team"]
y = y.astype('int')

In [ ]:
#separamos train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
score = logreg.score(X_train, y_train)
score2 = logreg.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

In [ ]:
ranking = pd.read_csv('fifa_ranking-2022-10-06.csv')
fixtures = pd.read_csv('results.csv')
rankings = ranking.drop_duplicates(subset='country_full',)
pred_set = []

In [ ]:
fixtures.insert(1, 'first_position', fixtures['Home Team'].map(rankings.set_index('country_full', verify_integrity= True)['rank']))

In [ ]:
fixtures.insert(2, 'second position', fixtures['Away Team'].map(rankings.set_index('country_full')['rank']))

In [ ]:
fixtures = fixtures.iloc[:48, :]

In [ ]:
for index, row in fixtures.iterrows():
    if row['first_position'] > row['second position']:
        pred_set.append({'home_team': row['Away Team'], 'away_team': row['Home Team'], 'winning_team': None})
    else:
        pred_set.append({'home_team': row['Home Team'], 'away_team': row['Away Team'], 'winning_team': None})

pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set

In [ ]:
print(pred_set)

In [ ]:
pred_set_1 = pd.get_dummies(pred_set, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])
print(pred_set_1)

In [ ]:
missing_cols = set(final.columns) - set(pred_set_1.columns)
for c in missing_cols:
    pred_set_1[c] = 0
pred_set_1 = pred_set_1[final.columns]

pred_set_1 = pred_set_1.drop(['winning_team'], axis=1)
pred_set_1.head()

In [ ]:
#matches de grupos
predictions = logreg.predict(pred_set_1)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 2:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    elif predictions[i] == 1:
        print("Draw")
    elif predictions[i] == 0:
        print("Winner:" + backup_pred_set.iloc[i, 1])
    print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ', '%3f'%(logreg.predict_proba(pred_set_1)[i][2]))
    print('Probability of Draw: ', '%3f'%(logreg.predict_proba(pred_set_1)[i][1]))
    print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%3f'%(logreg.predict_proba(pred_set_1)[i][0]))
    print("")

In [ ]:
#lista de tuplas que pasan
group_16 = [('Netherlands', 'Wales'), ('Argentina', 'Denmark'), ('France', 'Mexico'), ('England', 'Ecuador'), ('Germany', 'Croatia'), ('Brazil', 'Portugal'), ('Belgium', 'Spain'), ('Uruguay','Switzerland')]

In [ ]:
def clean_and_predict(matches, ranking, final, logreg):
    positions = []
    
    for match in matches:
        positions.append(rankings.loc[rankings['country_full'] == match[0], 'rank'].iloc[0])
        positions.append(rankings.loc[rankings['country_full'] == match[1], 'rank'].iloc[0])
        
    pred_set = []
    
    i = 0
    j = 0
    
    while i < len(positions):
        dict1 = {}
        
        if positions[i] < positions[i + 1]:
            dict1.update({'home_team': matches[j][0], 'away_team': matches[j][1]})
        else:
            dict1.update({'home_team': matches[j][1], 'away_team': matches[j][0]})
            
        pred_set.append(dict1)
        i += 2
        j += 1
        
    
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set
    
    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    # Remove winning team column
    pred_set = pred_set.drop(['winning_team'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 2:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        elif predictions[i] == 1:
            print("Draw")
        elif predictions[i] == 0:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ' , '%.3f'%(logreg.predict_proba(pred_set)[i][2]))
        print('Probability of Draw: ', '%.3f'%(logreg.predict_proba(pred_set)[i][1])) 
        print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set)[i][0]))
        print("")

In [ ]:
clean_and_predict(group_16, ranking, final, logreg)


In [ ]:
#lista de cuartos de final
quarters = [('Argentina', 'Germany'), ('Netherlands', 'Brazil'), ('France', 'England'), ('Spain', 'Switzerland')]

In [ ]:
clean_and_predict(quarters, ranking, final, logreg)

In [ ]:
#lista de semis
semi = [('Germany', 'England'), ('Brazil', 'Spain')] 

In [ ]:
clean_and_predict(semi, ranking, final, logreg)


In [ ]:
# Final
finals = [('Brazil', 'Germany')]

In [ ]:
clean_and_predict(finals, ranking, final, logreg)
